In [6]:
# ensemble_inference.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import timm
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm
import numpy as np

# --- 1. 설정 (Configuration) ---
# --- 공통 파라미터 ---
NUM_CLASSES = 7        # 모델 학습 시 사용된 클래스 수
# CRITICAL! 학습 시 순서와 정확히 일치해야 함
CLASS_NAMES = ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']
if len(CLASS_NAMES) != NUM_CLASSES:
    raise ValueError(f"CLASS_NAMES의 개수({len(CLASS_NAMES)})가 NUM_CLASSES({NUM_CLASSES})와 일치하지 않습니다.")

TEST_CSV_PATH = r'/home/metaai2/workspace/limseunghwan/open/test.csv'    # test.csv 경로
IMAGE_BASE_DIR = r'/home/metaai2/workspace/limseunghwan/open'           # 이미지 기본 디렉토리
OUTPUT_CSV_DIR = './submissions_ensemble' # 출력 CSV 저장 디렉토리

# --- 추론 파라미터 ---
# BATCH_SIZE_INFERENCE는 모델 중 가장 큰 모델에 맞춰 결정됩니다.
# 여기서는 공통된 값을 사용합니다. 모델 크기 차이가 크면 조정하세요.
BATCH_SIZE_INFERENCE = 4 # GPU 메모리에 따라 조정 (가장 큰 모델 기준)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS_INFERENCE = os.cpu_count() // 2 if os.cpu_count() else 4

# --- 모델별 설정 ---
# TODO: 중요! 실제 모델 세부 정보로 업데이트하세요.
MODEL_CONFIGS = [
    {
        "name": "MaxViT_XLarge_384_epoch19", # 이 모델에 대한 설명적인 이름
        "architecture": 'maxvit_xlarge_tf_384.in21k_ft_in1k',
        "saved_path": './saved_models_maxvit_xlarge_384/maxvit_xlarge_tf_384_epoch19_f1_0.9294.pth',
        "img_size": 384,
        "weight": 0.65 # 선택 사항: 소프트 보팅 시 가중 평균용
    },
    {
        "name": "ConvNeXt_Large_384_epoch20",
        "architecture": 'convnext_large.fb_in22k_ft_in1k_384',
        "saved_path": './saved_models_convnext/convnext_large_epoch19_f1_0.9121.pth', # 예시 경로!
        "img_size": 384,
        "weight": 0.5 # 선택 사항
    },
    # 필요한 경우 여기에 더 많은 모델 설정을 추가하세요.
]

# --- 무결성 검사: 모든 모델은 단일 데이터로더를 위해 동일한 이미지 크기를 사용하는 것이 이상적입니다 ---
# IMG_SIZE가 다르면 여러 데이터로더를 사용하거나 모델별로 즉석에서 크기를 조정하는 등 더 복잡한 설정이 필요합니다.
# 이 스크립트에서는 데이터로더를 위해 목록의 *첫 번째* 모델의 IMG_SIZE를 사용한다고 가정합니다.
# MODEL_CONFIGS의 모든 `img_size`가 동일한 것이 가장 좋습니다.
first_model_img_size = MODEL_CONFIGS[0]["img_size"]
for config in MODEL_CONFIGS:
    if config["img_size"] != first_model_img_size:
        print(f"경고: 모델 {config['name']}의 img_size({config['img_size']})가 첫 번째 모델의 img_size({first_model_img_size})와 다릅니다. "
              f"데이터로더는 {first_model_img_size}를 사용합니다. 이로 인해 {config['name']}의 성능이 저하될 수 있습니다.")
        # 이것이 의도된 것이 아니라면 오류를 발생시키거나 여러 데이터로더를 구현하는 것을 고려하십시오.
IMG_SIZE = first_model_img_size # 데이터로더에는 첫 번째 모델의 이미지 크기를 사용

# --- 2. 추론용 데이터 변환 (Data Transformations for Inference) ---
# 하나의 변환을 만듭니다. 이상적으로는 모델 중 하나 또는 일반적인 변환을 기반으로 합니다.
# 목록의 첫 번째 모델에 대해 timm의 권장 설정을 사용해 보겠습니다.
print(f"IMG_SIZE={IMG_SIZE}에 대한 추론 변환 정의 중 (기준: {MODEL_CONFIGS[0]['name']})...")
try:
    # 공유 데이터로더를 위한 '모델 인식' 변환을 얻기 위한 약간의 편법으로
    # 첫 번째 유형의 더미 모델 인스턴스를 만들어 해당 data_config를 가져옵니다.
    temp_model_for_config = timm.create_model(
        MODEL_CONFIGS[0]["architecture"],
        pretrained=True, # 기본 설정을 로드하기 위해 pretrained=True 사용
        num_classes=NUM_CLASSES, # 설정에는 필수는 아니지만 좋은 습관
        img_size=IMG_SIZE
    )
    config = timm.data.resolve_data_config({}, model=temp_model_for_config)
    config['input_size'] = (3, IMG_SIZE, IMG_SIZE) # (C, H, W)
    inference_transform = timm.data.create_transform(**config, is_training=False)
    print(f"timm의 기본 변환 사용: Mean={config['mean']}, Std={config['std']}")
    del temp_model_for_config # 정리
except Exception as e:
    print(f"첫 번째 모델에 대한 timm 설정 가져오기 실패 ({e}). ImageNet 기본값을 사용하여 수동으로 변환 정의 중.")
    inference_transform = transforms.Compose([
        transforms.Resize(IMG_SIZE, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
print(f"공유 추론 변환: {inference_transform}")


# --- 3. 테스트 이미지를 위한 사용자 정의 데이터셋 및 데이터로더 ---
class TestImageDataset(Dataset):
    def __init__(self, csv_path, img_dir_root, transform=None):
        self.data_frame = pd.read_csv(csv_path)
        self.img_dir_root = img_dir_root
        self.transform = transform
        if 'img_path' not in self.data_frame.columns:
            print(f"경고: {csv_path}에서 'img_path' 열을 찾을 수 없습니다. 첫 번째 열('{self.data_frame.columns[0]}')에 이미지 경로가 있다고 가정합니다.")
            self.img_path_column = self.data_frame.columns[0]
        else:
            self.img_path_column = 'img_path'
        print(f"CSV에서 이미지 경로에 '{self.img_path_column}' 열 사용 중.")

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        relative_img_path = self.data_frame.loc[idx, self.img_path_column]
        full_img_path = os.path.join(self.img_dir_root, relative_img_path)
        try:
            image = Image.open(full_img_path).convert('RGB')
        except FileNotFoundError:
            print(f"오류: {full_img_path}에서 이미지를 찾을 수 없습니다.")
            raise
        except Exception as e:
            print(f"오류: {full_img_path} 이미지를 열 수 없습니다: {e}")
            raise
        if self.transform:
            image = self.transform(image)
        return image, relative_img_path

print(f"\nCSV에서 테스트 데이터 로드 중: {TEST_CSV_PATH}")
try:
    test_dataset = TestImageDataset(csv_path=TEST_CSV_PATH,
                                    img_dir_root=IMAGE_BASE_DIR,
                                    transform=inference_transform)
    if len(test_dataset) == 0:
        print(f"오류: {TEST_CSV_PATH}에서 이미지를 찾을 수 없습니다.")
        exit()
    test_loader = DataLoader(test_dataset,
                             batch_size=BATCH_SIZE_INFERENCE,
                             shuffle=False,
                             num_workers=NUM_WORKERS_INFERENCE,
                             pin_memory=True)
    print(f"{len(test_dataset)}개의 이미지로 데이터로더를 성공적으로 생성했습니다.")
except FileNotFoundError:
    print(f"오류: {TEST_CSV_PATH}에서 테스트 CSV 파일을 찾을 수 없습니다.")
    exit()
except Exception as e:
    print(f"오류: 테스트 데이터셋 또는 데이터로더를 생성할 수 없습니다: {e}")
    exit()


# --- 4. 모델 로딩 및 예측 함수 ---
def load_model_from_config(model_conf):
    print(f"모델 아키텍처 로드 중: {model_conf['architecture']}")

    # 모델 아키텍처에 따라 img_size 인자 전달 여부 결정
    if 'maxvit' in model_conf['architecture'].lower(): # MaxViT 계열 모델인 경우
        model = timm.create_model(
            model_conf['architecture'],
            pretrained=False,
            num_classes=NUM_CLASSES,
            img_size=model_conf['img_size'] # MaxViT는 img_size를 받음
        )
    elif 'convnext' in model_conf['architecture'].lower(): # ConvNeXt 계열 모델인 경우
        model = timm.create_model(
            model_conf['architecture'],
            pretrained=False,
            num_classes=NUM_CLASSES
            # ConvNeXt는 일반적으로 img_size 인자를 직접 받지 않음
        )
    else: # 다른 모델 아키텍처의 경우 (기본적으로 img_size를 전달하지 않거나, 필요시 추가 조건)
        print(f"주의: {model_conf['architecture']} 아키텍처에 대한 img_size 처리 규칙이 명시되지 않았습니다. img_size 인자 없이 로드를 시도합니다.")
        # 대부분의 timm 모델은 img_size를 필수로 요구하지 않거나, 아키텍처 이름에 크기가 포함됨.
        # 특정 모델이 img_size를 요구하고 위 조건에 해당하지 않는다면, 여기에 elif로 추가해야 함.
        # 예: elif 'some_other_model_prefix' in model_conf['architecture'].lower():
        #         model = timm.create_model(..., img_size=model_conf['img_size'])
        model = timm.create_model(
            model_conf['architecture'],
            pretrained=False,
            num_classes=NUM_CLASSES
        )

    print(f"학습된 가중치 로드 중: {model_conf['saved_path']}")
    if not os.path.exists(model_conf['saved_path']):
        print(f"오류: 모델 가중치 파일 {model_conf['saved_path']}을(를) 찾을 수 없습니다.") # 경로 변수명 수정
        raise FileNotFoundError(f"모델 가중치를 찾을 수 없음: {model_conf['saved_path']}")
    try:
        model.load_state_dict(torch.load(model_conf['saved_path'], map_location=DEVICE))
        print(f"{model_conf['name']}의 가중치를 {DEVICE}에 성공적으로 로드했습니다.")
    except Exception as e:
        print(f"오류: {model_conf['name']}의 가중치를 로드할 수 없습니다: {e}")
        raise
    model = model.to(DEVICE)
    model.eval()
    return model

def get_probabilities(model, loader, device, model_name="Model"):
    model.eval()
    all_probs_list = []
    all_original_filenames_list = [] # 순서와 내용 일관성 보장을 위해

    with torch.no_grad():
        for images_batch, filenames_batch in tqdm(loader, desc=f"{model_name}으로 예측 중"):
            images_batch = images_batch.to(device)

            # 모델이 예상하는 img_size가 데이터로더의 img_size와 다르면 여기서 크기 조정 (더 복잡함)
            # 현재는 이전 검사 / 공유 IMG_SIZE로 인해 일치한다고 가정
            # if model.img_size != images_batch.shape[-1]: # 확인을 위한 의사 코드
            #    images_batch = transforms.functional.resize(images_batch, [model.img_size, model.img_size])

            outputs = model(images_batch)
            # 소프트맥스 확률 얻기
            probs = torch.softmax(outputs, dim=1)
            all_probs_list.append(probs.cpu()) # CPU에 저장
            all_original_filenames_list.extend(list(filenames_batch))

    all_probs_tensor = torch.cat(all_probs_list, dim=0)
    return all_probs_tensor.numpy(), all_original_filenames_list


# --- 5. 모든 모델에 대한 추론 실행 및 앙상블 ---
if __name__ == "__main__":
    print("\n앙상블 추론 프로세스 시작 중...")

    all_model_probs = [] # 각 모델의 확률 배열(이미지 수 x 클래스 수)을 저장할 리스트
    model_weights = []   # 가중 평균용 (지정된 경우)
    image_paths_from_loader = None # 첫 번째 모델 실행에서 이미지 경로를 저장하기 위함

    for i, model_conf in enumerate(MODEL_CONFIGS):
        print(f"\n--- 모델 처리 중: {model_conf['name']} ---")
        try:
            model = load_model_from_config(model_conf)
            probs_np, current_paths = get_probabilities(model, test_loader, DEVICE, model_conf['name'])
            all_model_probs.append(probs_np)

            if 'weight' in model_conf:
                model_weights.append(model_conf['weight'])
            else:
                model_weights.append(1.0) # 기본 가중치는 1.0

            if image_paths_from_loader is None:
                image_paths_from_loader = current_paths
            elif image_paths_from_loader != current_paths:
                # 데이터로더가 일관적이라면 이상적으로는 발생하지 않아야 함
                print("경고: 모델 예측 간 이미지 경로 순서 불일치. 문제가 있을 수 있습니다.")
                # 견고성을 위해 경로를 기반으로 재정렬을 시도할 수 있지만 복잡합니다.
                # 현재는 데이터로더의 순서가 항상 동일하다고 가정합니다.
                pass
            del model # 메모리 해제
            torch.cuda.empty_cache() # CUDA 캐시 비우기
        except Exception as e:
            print(f"오류: 모델 {model_conf['name']} 처리 중 오류 발생: {e}. 이 모델은 앙상블에서 제외합니다.")
            # 선택적으로, 오류 전에 가중치가 추가된 경우 해당 가중치를 제거하거나 나중에 처리
            if len(model_weights) > len(all_model_probs):
                model_weights.pop()


    if not all_model_probs:
        print("오류: 성공적으로 처리된 모델이 없습니다. 앙상블을 생성할 수 없습니다. 종료합니다.")
        exit()

    # --- 앙상블 예측 (소프트 보팅, 선택적 가중치 사용) ---
    print("\n소프트 보팅을 사용하여 예측 앙상블 중...")

    # 가중치 정규화 (이미 합이 1이 아니면 합이 1이 되도록)
    total_weight = sum(model_weights)
    if total_weight == 0: # 적어도 하나의 모델이 성공했다면 발생하지 않아야 함
        print("경고: 총 가중치가 0입니다. 동일한 가중치를 사용합니다.")
        normalized_weights = [1.0/len(all_model_probs)] * len(all_model_probs)
    else:
        normalized_weights = [w / total_weight for w in model_weights]

    # 확률의 가중 평균
    # all_model_probs가 numpy 배열인지 확인
    ensembled_probs_sum = np.zeros_like(all_model_probs[0]) # 첫 번째 모델 확률의 형태로 초기화
    for i, probs_array in enumerate(all_model_probs):
        ensembled_probs_sum += probs_array * normalized_weights[i]

    # 앙상블된 확률에서 최종 예측 인덱스
    final_predicted_indices = np.argmax(ensembled_probs_sum, axis=1)

    # 인덱스를 클래스 이름으로 변환
    ensembled_class_names = [CLASS_NAMES[idx] for idx in final_predicted_indices]

    # --- 6. 제출 파일 생성 및 저장 ---
    print("\n제출 파일 준비 중...")
    original_test_df = pd.read_csv(TEST_CSV_PATH)

    # 데이터로더에서 반환된 이미지 경로와 앙상블된 예측을 매핑
    prediction_map = {
        os.path.normpath(path): pred_label
        for path, pred_label in zip(image_paths_from_loader, ensembled_class_names)
    }

    csv_img_path_col = test_dataset.img_path_column
    mapped_predictions = original_test_df[csv_img_path_col].apply(
        lambda x: prediction_map.get(os.path.normpath(x))
    )

    submission_df = pd.DataFrame()
    submission_df['ID'] = original_test_df[csv_img_path_col].apply(
        lambda x: os.path.splitext(os.path.basename(x))[0]
    )
    submission_df['rock_type'] = mapped_predictions

    if submission_df['rock_type'].isnull().any():
        num_null = submission_df['rock_type'].isnull().sum()
        print(f"경고: CSV의 {num_null}개 이미지가 예측을 받지 못했습니다. "
              "누락된 이미지 파일 또는 매핑 중 오류 때문일 수 있습니다.")
        # 필요한 경우 NaN 값을 채웁니다. 예: 'Etc' 클래스 또는 가장 빈번한 클래스 (허용되는 경우)
        # submission_df['rock_type'].fillna('Etc', inplace=True)
        # print(f"{num_null}개의 NaN을 'Etc'로 채웠습니다.")

    os.makedirs(OUTPUT_CSV_DIR, exist_ok=True)
    # 성공적으로 로드된 모델의 이름만 사용
    ensemble_model_names_list = []
    for cfg in MODEL_CONFIGS:
        # Check if model was successfully processed by seeing if its probs are in all_model_probs
        # This requires matching by some unique identifier, e.g. its original config index or name if names are unique
        # For simplicity, let's assume order in MODEL_CONFIGS matches order in all_model_probs if no failures
        # Or, more robustly, only include names of models whose paths exist and were attempted.
        if os.path.exists(cfg["saved_path"]): # Simple check, assumes path existence means it was part of the attempt
            cleaned_name = cfg["name"].split('_epoch')[0].replace('_384','')
            ensemble_model_names_list.append(cleaned_name)

    ensemble_model_names = "_".join(ensemble_model_names_list)

    if not all_model_probs: # 모든 모델 로딩 실패 시
        ensemble_model_names = "ENSEMBLE_FAILED"
    elif len(all_model_probs) < len(MODEL_CONFIGS): # 일부 모델만 성공
         ensemble_model_names += "_PARTIAL"


    output_csv_filename = f"submission_ensemble_{ensemble_model_names}.csv"
    final_output_csv_path = os.path.join(OUTPUT_CSV_DIR, output_csv_filename)

    submission_df.to_csv(final_output_csv_path, index=False)
    print(f"\n앙상블 추론 완료. 예측 결과 저장 위치: {final_output_csv_path}")
    print(f"제출 파일 샘플:\n{submission_df.head()}")

IMG_SIZE=384에 대한 추론 변환 정의 중 (기준: MaxViT_XLarge_384_epoch19)...
timm의 기본 변환 사용: Mean=(0.5, 0.5, 0.5), Std=(0.5, 0.5, 0.5)
공유 추론 변환: Compose(
    Resize(size=(384, 384), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(384, 384))
    MaybeToTensor()
    Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
)

CSV에서 테스트 데이터 로드 중: /home/metaai2/workspace/limseunghwan/open/test.csv
CSV에서 이미지 경로에 'img_path' 열 사용 중.
95006개의 이미지로 데이터로더를 성공적으로 생성했습니다.

앙상블 추론 프로세스 시작 중...

--- 모델 처리 중: MaxViT_XLarge_384_epoch19 ---
모델 아키텍처 로드 중: maxvit_xlarge_tf_384.in21k_ft_in1k
학습된 가중치 로드 중: ./saved_models_maxvit_xlarge_384/maxvit_xlarge_tf_384_epoch19_f1_0.9294.pth
MaxViT_XLarge_384_epoch19의 가중치를 cuda에 성공적으로 로드했습니다.


MaxViT_XLarge_384_epoch19으로 예측 중:  78%|███████▊  | 18568/23752 [31:40<08:50,  9.77it/s]


KeyboardInterrupt: 

In [ ]:
# ensemble_inference.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import timm
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm
import numpy as np

# --- 1. 설정 (Configuration) ---
# --- 공통 파라미터 ---
NUM_CLASSES = 7        # 모델 학습 시 사용된 클래스 수
# CRITICAL! 학습 시 순서와 정확히 일치해야 함
CLASS_NAMES = ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']
if len(CLASS_NAMES) != NUM_CLASSES:
    raise ValueError(f"CLASS_NAMES의 개수({len(CLASS_NAMES)})가 NUM_CLASSES({NUM_CLASSES})와 일치하지 않습니다.")

TEST_CSV_PATH = r'/home/metaai2/workspace/limseunghwan/open/test.csv'    # test.csv 경로
IMAGE_BASE_DIR = r'/home/metaai2/workspace/limseunghwan/open'           # 이미지 기본 디렉토리
OUTPUT_CSV_DIR = './submissions_ensemble' # 출력 CSV 저장 디렉토리

# --- 추론 파라미터 ---
BATCH_SIZE_INFERENCE = 4 # GPU 메모리에 따라 조정 (가장 큰 모델 기준)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS_INFERENCE = os.cpu_count() // 2 if os.cpu_count() is not None else 4 # os.cpu_count() None 반환 가능성 처리

# --- 모델별 설정 ---
MODEL_CONFIGS = [
    {
        "name": "MaxViT_XLarge_384_epoch19",
        "architecture": 'maxvit_xlarge_tf_384.in21k_ft_in1k',
        "saved_path": './saved_models_maxvit_xlarge_384/maxvit_xlarge_tf_384_epoch19_f1_0.9294.pth',
        "img_size": 384,
        "weight": 0.75
    },
    {
        "name": "ConvNeXt_Large_384_epoch20",
        "architecture": 'convnext_large.fb_in22k_ft_in1k_384',
        "saved_path": './saved_models_convnext/convnext_large_epoch19_f1_0.9121.pth',
        "img_size": 384,
        "weight": 0.5
    },
    {
        "name": "Swin_Large_384_Best", # Swin Transformer 모델 이름
        "architecture": 'swin_large_patch4_window12_384.ms_in22k_ft_in1k', # Swin Transformer 아키텍처
        "saved_path": './best_swin_large_model_test_limited_batches.pth', # Swin Transformer 가중치 경로
        "img_size": 384, # Swin Transformer 이미지 크기
        "weight": 0.5 # Swin Transformer 앙상블 가중치 (예시, 조정 필요)
    }
]

# --- 무결성 검사: 모든 모델은 단일 데이터로더를 위해 동일한 이미지 크기를 사용하는 것이 이상적입니다 ---
first_model_img_size = MODEL_CONFIGS[0]["img_size"]
for config in MODEL_CONFIGS:
    if config["img_size"] != first_model_img_size:
        print(f"경고: 모델 {config['name']}의 img_size({config['img_size']})가 첫 번째 모델의 img_size({first_model_img_size})와 다릅니다. "
              f"데이터로더는 {first_model_img_size}를 사용합니다. 이로 인해 {config['name']}의 성능이 저하될 수 있습니다.")
IMG_SIZE = first_model_img_size

# --- 2. 추론용 데이터 변환 (Data Transformations for Inference) ---
print(f"IMG_SIZE={IMG_SIZE}에 대한 추론 변환 정의 중 (기준: {MODEL_CONFIGS[0]['name']})...")
try:
    temp_model_for_config = timm.create_model(
        MODEL_CONFIGS[0]["architecture"],
        pretrained=True,
        num_classes=NUM_CLASSES,
        img_size=IMG_SIZE # img_size 명시적 전달 (MaxViT의 경우 필요할 수 있음)
    )
    data_config = timm.data.resolve_data_config({}, model=temp_model_for_config)
    # data_config['input_size'] = (3, IMG_SIZE, IMG_SIZE) # timm이 알아서 설정하므로 중복될 수 있음
    inference_transform = timm.data.create_transform(**data_config, is_training=False)
    print(f"timm의 기본 변환 사용: Mean={data_config['mean']}, Std={data_config['std']}, Input_Size={data_config['input_size']}")
    del temp_model_for_config
except Exception as e:
    print(f"첫 번째 모델에 대한 timm 설정 가져오기 실패 ({e}). ImageNet 기본값을 사용하여 수동으로 변환 정의 중.")
    inference_transform = transforms.Compose([
        transforms.Resize(IMG_SIZE, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
print(f"공유 추론 변환: {inference_transform}")


# --- 3. 테스트 이미지를 위한 사용자 정의 데이터셋 및 데이터로더 ---
class TestImageDataset(Dataset):
    def __init__(self, csv_path, img_dir_root, transform=None):
        self.data_frame = pd.read_csv(csv_path)
        self.img_dir_root = img_dir_root
        self.transform = transform
        if 'img_path' not in self.data_frame.columns:
            print(f"경고: {csv_path}에서 'img_path' 열을 찾을 수 없습니다. 첫 번째 열('{self.data_frame.columns[0]}')에 이미지 경로가 있다고 가정합니다.")
            self.img_path_column = self.data_frame.columns[0]
        else:
            self.img_path_column = 'img_path'
        print(f"CSV에서 이미지 경로에 '{self.img_path_column}' 열 사용 중.")

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        relative_img_path = self.data_frame.loc[idx, self.img_path_column]
        full_img_path = os.path.join(self.img_dir_root, relative_img_path)
        try:
            image = Image.open(full_img_path).convert('RGB')
        except FileNotFoundError:
            print(f"오류: {full_img_path}에서 이미지를 찾을 수 없습니다.")
            raise
        except Exception as e:
            print(f"오류: {full_img_path} 이미지를 열 수 없습니다: {e}")
            raise
        if self.transform:
            image = self.transform(image)
        return image, relative_img_path

print(f"\nCSV에서 테스트 데이터 로드 중: {TEST_CSV_PATH}")
try:
    test_dataset = TestImageDataset(csv_path=TEST_CSV_PATH,
                                    img_dir_root=IMAGE_BASE_DIR,
                                    transform=inference_transform)
    if len(test_dataset) == 0:
        print(f"오류: {TEST_CSV_PATH}에서 이미지를 찾을 수 없습니다.")
        exit()
    test_loader = DataLoader(test_dataset,
                             batch_size=BATCH_SIZE_INFERENCE,
                             shuffle=False,
                             num_workers=NUM_WORKERS_INFERENCE,
                             pin_memory=True)
    print(f"{len(test_dataset)}개의 이미지로 데이터로더를 성공적으로 생성했습니다.")
except FileNotFoundError:
    print(f"오류: {TEST_CSV_PATH}에서 테스트 CSV 파일을 찾을 수 없습니다.")
    exit()
except Exception as e:
    print(f"오류: 테스트 데이터셋 또는 데이터로더를 생성할 수 없습니다: {e}")
    exit()


# --- 4. 모델 로딩 및 예측 함수 ---
def load_model_from_config(model_conf):
    print(f"모델 아키텍처 로드 중: {model_conf['architecture']}")
    architecture_lower = model_conf['architecture'].lower()

    # img_size를 필요로 하는 모델이나, 아키텍처 문자열에 크기가 명시되지 않은 경우를 위한 처리
    # timm 대부분 모델은 아키텍처 이름에 크기가 명시되어 있으면 img_size 인자가 필수는 아님.
    # 하지만 명시적으로 전달하는 것이 더 안전할 수 있음.
    if 'maxvit' in architecture_lower : # MaxViT는 img_size를 명시적으로 받는 것이 좋음
        model = timm.create_model(
            model_conf['architecture'],
            pretrained=False,
            num_classes=NUM_CLASSES,
            img_size=model_conf['img_size']
        )
    # ConvNeXt의 경우 아키텍처 이름에 img_size가 포함되어 있음 (e.g., convnext_large_in22k_ft_in1k_384)
    # Swin Transformer의 경우도 아키텍처 이름에 img_size가 포함되어 있음 (e.g., swin_large_patch4_window12_384)
    # 따라서 이들은 img_size 인자 없이도 잘 로드될 수 있음.
    elif 'convnext' in architecture_lower or 'swin' in architecture_lower:
         model = timm.create_model(
            model_conf['architecture'],
            pretrained=False,
            num_classes=NUM_CLASSES
            # img_size=model_conf['img_size'] # 필요하다면 추가할 수 있으나, 보통 아키텍처 이름으로 충분
        )
    else:
        print(f"주의: {model_conf['architecture']} 아키텍처에 대한 img_size 처리 규칙이 명시되지 않았습니다. img_size 인자 없이 로드를 시도합니다.")
        model = timm.create_model(
            model_conf['architecture'],
            pretrained=False,
            num_classes=NUM_CLASSES
        )

    print(f"학습된 가중치 로드 중: {model_conf['saved_path']}")
    if not os.path.exists(model_conf['saved_path']):
        print(f"오류: 모델 가중치 파일 {model_conf['saved_path']}을(를) 찾을 수 없습니다.")
        raise FileNotFoundError(f"모델 가중치를 찾을 수 없음: {model_conf['saved_path']}")
    try:
        # 모델 가중치 로드 시 map_location을 사용하여 CPU 또는 GPU로 로드 가능
        state_dict = torch.load(model_conf['saved_path'], map_location=torch.device('cpu'))
        # 만약 state_dict이 'model', 'state_dict' 등의 키를 가지고 있다면, 실제 가중치를 추출해야 할 수 있음
        # 예: if 'state_dict' in state_dict: state_dict = state_dict['state_dict']
        # 예: if 'model' in state_dict: state_dict = state_dict['model']
        model.load_state_dict(state_dict)
        print(f"{model_conf['name']}의 가중치를 성공적으로 로드했습니다.")
    except Exception as e:
        print(f"오류: {model_conf['name']}의 가중치를 로드할 수 없습니다: {e}")
        # 가끔 DataParallel 등으로 저장된 모델은 키 앞에 'module.' 접두사가 붙을 수 있음
        # 이 경우, 접두사 제거 후 다시 시도하는 로직 추가 가능
        try:
            print("키 앞에 'module.' 접두사가 있는지 확인 후 다시 로드 시도 중...")
            new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
            model.load_state_dict(new_state_dict)
            print(f"{model_conf['name']}의 가중치('module.' 제거 후)를 성공적으로 로드했습니다.")
        except Exception as e2:
            print(f"오류: 'module.' 접두사 제거 후에도 {model_conf['name']}의 가중치를 로드할 수 없습니다: {e2}")
            raise e # 원본 오류를 다시 발생시킴
            
    model = model.to(DEVICE)
    model.eval()
    return model

def get_probabilities(model, loader, device, model_name="Model"):
    model.eval()
    all_probs_list = []
    all_original_filenames_list = []

    with torch.no_grad():
        for images_batch, filenames_batch in tqdm(loader, desc=f"{model_name}으로 예측 중"):
            images_batch = images_batch.to(device)
            outputs = model(images_batch)
            probs = torch.softmax(outputs, dim=1)
            all_probs_list.append(probs.cpu())
            all_original_filenames_list.extend(list(filenames_batch))

    all_probs_tensor = torch.cat(all_probs_list, dim=0)
    return all_probs_tensor.numpy(), all_original_filenames_list


# --- 5. 모든 모델에 대한 추론 실행 및 앙상블 ---
if __name__ == "__main__":
    print("\n앙상블 추론 프로세스 시작 중...")
    print(f"사용 디바이스: {DEVICE}")

    all_model_probs = []
    model_weights = []
    image_paths_from_loader = None
    processed_model_configs = [] # 성공적으로 처리된 모델의 설정을 저장

    for i, model_conf in enumerate(MODEL_CONFIGS):
        print(f"\n--- 모델 처리 중: {model_conf['name']} ---")
        try:
            model = load_model_from_config(model_conf)
            probs_np, current_paths = get_probabilities(model, test_loader, DEVICE, model_conf['name'])
            all_model_probs.append(probs_np)
            processed_model_configs.append(model_conf) # 성공 시 추가

            if 'weight' in model_conf:
                model_weights.append(model_conf['weight'])
            else:
                model_weights.append(1.0)

            if image_paths_from_loader is None:
                image_paths_from_loader = current_paths
            elif image_paths_from_loader != current_paths:
                print("경고: 모델 예측 간 이미지 경로 순서 불일치. 문제가 있을 수 있습니다.")
            
            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        except FileNotFoundError as e:
            print(f"오류: 모델 {model_conf['name']}의 가중치 파일을 찾을 수 없습니다: {e}. 이 모델은 앙상블에서 제외합니다.")
        except Exception as e:
            print(f"오류: 모델 {model_conf['name']} 처리 중 오류 발생: {e}. 이 모델은 앙상블에서 제외합니다.")
            # 이미 model_weights에 추가되었다면 제거 (오류 발생 전에 추가되므로)
            # 이 부분은 현재 로직상 불필요 (오류 시 model_weights에 추가되지 않음)

    if not all_model_probs:
        print("오류: 성공적으로 처리된 모델이 없습니다. 앙상블을 생성할 수 없습니다. 종료합니다.")
        exit()

    # --- 앙상블 예측 (소프트 보팅, 선택적 가중치 사용) ---
    print("\n소프트 보팅을 사용하여 예측 앙상블 중...")

    # 가중치 정규화 (이미 합이 1이 아니면 합이 1이 되도록)
    # 성공적으로 처리된 모델들의 가중치만 사용
    if sum(model_weights) == 0 and len(model_weights) > 0 : # 모든 가중치가 0인 극단적 경우 (하지만 모델은 있음)
        print("경고: 총 가중치가 0이지만 처리된 모델이 있습니다. 동일한 가중치를 사용합니다.")
        normalized_weights = [1.0/len(all_model_probs)] * len(all_model_probs)
    elif sum(model_weights) > 0 :
        total_weight = sum(model_weights)
        normalized_weights = [w / total_weight for w in model_weights]
    else: # all_model_probs는 있지만 model_weights가 비어있거나 (이론상 불가능), 다른 문제
        print("경고: 가중치 설정에 문제가 있습니다. 동일한 가중치를 사용합니다.")
        normalized_weights = [1.0/len(all_model_probs)] * len(all_model_probs)


    ensembled_probs_sum = np.zeros_like(all_model_probs[0])
    print("앙상블에 사용된 모델 및 가중치:")
    for i, probs_array in enumerate(all_model_probs):
        # processed_model_configs[i]['name']을 사용하여 현재 모델의 이름 표시
        print(f"- 모델: {processed_model_configs[i]['name']}, 정규화된 가중치: {normalized_weights[i]:.4f}")
        ensembled_probs_sum += probs_array * normalized_weights[i]

    final_predicted_indices = np.argmax(ensembled_probs_sum, axis=1)
    ensembled_class_names = [CLASS_NAMES[idx] for idx in final_predicted_indices]

    # --- 6. 제출 파일 생성 및 저장 ---
    print("\n제출 파일 준비 중...")
    original_test_df = pd.read_csv(TEST_CSV_PATH)

    prediction_map = {
        os.path.normpath(path): pred_label
        for path, pred_label in zip(image_paths_from_loader, ensembled_class_names)
    }

    csv_img_path_col = test_dataset.img_path_column
    mapped_predictions = original_test_df[csv_img_path_col].apply(
        lambda x: prediction_map.get(os.path.normpath(x))
    )

    submission_df = pd.DataFrame()
    # 원본 CSV의 'ID' 컬럼이 있다면 사용, 없다면 파일명에서 추출
    if 'ID' in original_test_df.columns:
        submission_df['ID'] = original_test_df['ID']
    else:
        submission_df['ID'] = original_test_df[csv_img_path_col].apply(
            lambda x: os.path.splitext(os.path.basename(x))[0]
        )
    submission_df['rock_type'] = mapped_predictions

    if submission_df['rock_type'].isnull().any():
        num_null = submission_df['rock_type'].isnull().sum()
        print(f"경고: CSV의 {num_null}개 이미지가 예측을 받지 못했습니다. "
              "누락된 이미지 파일 또는 매핑 중 오류 때문일 수 있습니다.")
        # submission_df['rock_type'].fillna('Etc', inplace=True) # 필요시 주석 해제
        # print(f"{num_null}개의 NaN을 'Etc'로 채웠습니다.")

    os.makedirs(OUTPUT_CSV_DIR, exist_ok=True)
    
    # 성공적으로 로드된 모델의 이름만 사용
    ensemble_model_names_list = []
    for cfg in processed_model_configs: # 성공적으로 처리된 모델 설정만 사용
        cleaned_name = cfg["name"].replace('_epoch', '').replace('_384','').replace('_f1_','_') # 이름 정제
        # 특정 패턴 제거 (예: _0.xxxx 소수점 부분)
        cleaned_name = '_'.join(cleaned_name.split('_')[:3]) # 이름이 너무 길어지는 것 방지 (예시)
        ensemble_model_names_list.append(cleaned_name)

    ensemble_model_names_str = "_".join(sorted(list(set(ensemble_model_names_list)))) # 중복 제거 및 정렬

    if not ensemble_model_names_str:
        ensemble_model_names_str = "ENSEMBLE_FAILED"
    elif len(processed_model_configs) < len(MODEL_CONFIGS):
         ensemble_model_names_str += "_PARTIAL"

    output_csv_filename = f"submission_ensemble_{ensemble_model_names_str}.csv"
    final_output_csv_path = os.path.join(OUTPUT_CSV_DIR, output_csv_filename)

    submission_df.to_csv(final_output_csv_path, index=False)
    print(f"\n앙상블 추론 완료. 예측 결과 저장 위치: {final_output_csv_path}")
    print(f"제출 파일 샘플:\n{submission_df.head()}")

/home/metaai2/miniconda3/envs/env312_cuda124_torch260/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


IMG_SIZE=384에 대한 추론 변환 정의 중 (기준: MaxViT_XLarge_384_epoch19)...
timm의 기본 변환 사용: Mean=(0.5, 0.5, 0.5), Std=(0.5, 0.5, 0.5), Input_Size=(3, 384, 384)
공유 추론 변환: Compose(
    Resize(size=(384, 384), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(384, 384))
    MaybeToTensor()
    Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
)

CSV에서 테스트 데이터 로드 중: /home/metaai2/workspace/limseunghwan/open/test.csv
CSV에서 이미지 경로에 'img_path' 열 사용 중.
95006개의 이미지로 데이터로더를 성공적으로 생성했습니다.

앙상블 추론 프로세스 시작 중...
사용 디바이스: cuda

--- 모델 처리 중: MaxViT_XLarge_384_epoch19 ---
모델 아키텍처 로드 중: maxvit_xlarge_tf_384.in21k_ft_in1k
학습된 가중치 로드 중: ./saved_models_maxvit_xlarge_384/maxvit_xlarge_tf_384_epoch19_f1_0.9294.pth
MaxViT_XLarge_384_epoch19의 가중치를 성공적으로 로드했습니다.


MaxViT_XLarge_384_epoch19으로 예측 중: 100%|██████████| 23752/23752 [40:24<00:00,  9.80it/s]



--- 모델 처리 중: ConvNeXt_Large_384_epoch20 ---
모델 아키텍처 로드 중: convnext_large.fb_in22k_ft_in1k_384
학습된 가중치 로드 중: ./saved_models_convnext/convnext_large_epoch19_f1_0.9121.pth
ConvNeXt_Large_384_epoch20의 가중치를 성공적으로 로드했습니다.


ConvNeXt_Large_384_epoch20으로 예측 중: 100%|██████████| 23752/23752 [11:46<00:00, 33.60it/s]



--- 모델 처리 중: Swin_Large_384_Best ---
모델 아키텍처 로드 중: swin_large_patch4_window12_384.ms_in22k_ft_in1k
학습된 가중치 로드 중: ./best_swin_large_model_test_limited_batches.pth
Swin_Large_384_Best의 가중치를 성공적으로 로드했습니다.


Swin_Large_384_Best으로 예측 중: 100%|██████████| 23752/23752 [15:21<00:00, 25.77it/s]



소프트 보팅을 사용하여 예측 앙상블 중...
앙상블에 사용된 모델 및 가중치:
- 모델: MaxViT_XLarge_384_epoch19, 정규화된 가중치: 0.4000
- 모델: ConvNeXt_Large_384_epoch20, 정규화된 가중치: 0.2667
- 모델: Swin_Large_384_Best, 정규화된 가중치: 0.3333

제출 파일 준비 중...

앙상블 추론 완료. 예측 결과 저장 위치: ./submissions_ensemble/submission_ensemble_ConvNeXt_Large20_MaxViT_XLarge19_Swin_Large_Best.csv
제출 파일 샘플:
           ID      rock_type
0  TEST_00000  Mud_Sandstone
1  TEST_00001  Mud_Sandstone
2  TEST_00002  Mud_Sandstone
3  TEST_00003        Granite
4  TEST_00004        Granite


In [2]:
# generate_oof_predictions_imagefolder.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset # Dataset 제거 (ImageFolder 사용)
from torchvision import transforms, datasets # datasets.ImageFolder 추가
import timm
# import pandas as pd # CSV 직접 사용 안 함
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import StratifiedKFold
from timm.data import resolve_data_config, create_transform

# --- 1. 설정 (Configuration) ---
# --- 공통 파라미터 ---
NUM_CLASSES = 7
# CLASS_NAMES는 ImageFolder가 자동으로 폴더 순서대로 인식하므로,
# ImageFolder 로드 후 실제 클래스 순서를 확인하고 일치시켜야 함
# 또는, train_meta_models.py에서 CLASS_NAMES 순서에 맞게 레이블을 인코딩할 때 사용
CLASS_NAMES_EXPECTED = ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']


# --- 데이터 경로 설정 (ImageFolder용) ---
TRAIN_DATA_ROOT_DIR = r"/home/metaai2/workspace/limseunghwan/open" # 학습/검증 데이터가 있는 최상위 디렉토리
# TRAIN_DATA_DIR = os.path.join(TRAIN_DATA_ROOT_DIR, "train") # 이전에 사용된 변수명 유지 또는 변경
# VAL_DATA_DIR = os.path.join(TRAIN_DATA_ROOT_DIR, "val") # 만약 train/val 분리된 경우

# OOF는 전체 학습 데이터에 대해 생성하므로, train 폴더만 사용한다고 가정
# 만약 train + val 데이터를 합쳐서 OOF를 만들고 싶다면, 두 ImageFolder를 ConcatDataset으로 합쳐야 함.
# 여기서는 TRAIN_DATA_DIR만 사용한다고 가정
TRAIN_IMAGE_FOLDER_PATH = os.path.join(TRAIN_DATA_ROOT_DIR, "train")


OOF_OUTPUT_DIR = './oof_predictions_imgfolder' # OOF 예측 및 레이블 저장 디렉토리

# --- 학습 및 OOF 생성 파라미터 ---
N_SPLITS = 5
RANDOM_STATE = 42
BATCH_SIZE_TRAIN = 8 # 제공된 설정값 사용
BATCH_SIZE_VAL = 16 # 검증은 더 크게 가능
EPOCHS_PER_FOLD = 5 # 제공된 설정값 EPOCHS = 20을 각 Fold에 분배하거나, Fold마다 동일하게 적용
                    # 여기서는 시연을 위해 5로 설정. 실제로는 더 길게 학습 필요
LEARNING_RATE = 1e-5 # 제공된 설정값 BASE_LR 사용
WEIGHT_DECAY = 1e-2
WARMUP_EPOCHS = 5 # 이 예제에서는 Warmup 스케줄러는 생략 (필요시 추가)
GRADIENT_CLIPPING = 1.0

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = os.cpu_count() // 2 if os.cpu_count() is not None else 4


# --- 기본 모델 설정 (OOF 생성 대상 모델) ---
# 제공해주신 설정에서는 단일 모델을 학습하는 형태였으나,
# OOF는 여러 기본 모델에 대해 만들 수 있으므로 리스트 형태로 유지
MODEL_CONFIGS = [
    {
        "name": "ConvNeXt_Large_384_ImageFolder", # OOF 파일명에 사용될 이름
        "architecture": 'convnext_large.fb_in22k_ft_in1k_384',
        "img_size": 384, # 모델 입력 이미지 크기
        "requires_img_size_arg": False # ConvNeXt는 아키텍처 이름에 크기 포함
    },
    # 여기에 다른 기본 모델들을 추가할 수 있습니다.
    # {
    #     "name": "MaxViT_XLarge_384_ImageFolder",
    #     "architecture": 'maxvit_xlarge_tf_384.in21k_ft_in1k',
    #     "img_size": 384,
    #     "requires_img_size_arg": True
    # },
]
IMG_SIZE_FOR_LOADER = MODEL_CONFIGS[0]["img_size"] # 첫 번째 모델 기준으로 변환 설정

# --- 2. 변환 정의 ---
try:
    temp_model_for_transform = timm.create_model(
        MODEL_CONFIGS[0]["architecture"], pretrained=True, num_classes=NUM_CLASSES
    )
    data_config = resolve_data_config({}, model=temp_model_for_transform)
    train_transform = create_transform(**data_config, is_training=True)
    val_transform = create_transform(**data_config, is_training=False)
    print(f"timm 기반 변환 사용. 학습: {train_transform}")
    print(f"timm 기반 변환 사용. 검증: {val_transform}")
    del temp_model_for_transform
except Exception as e:
    print(f"timm 설정 가져오기 실패 ({e}). ImageNet 기본값 사용.")
    train_transform = transforms.Compose([
        transforms.Resize((IMG_SIZE_FOR_LOADER, IMG_SIZE_FOR_LOADER)), # timm은 보통 RandomResizedCrop 사용
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # 가벼운 증강 추가
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transform = transforms.Compose([
        transforms.Resize((IMG_SIZE_FOR_LOADER, IMG_SIZE_FOR_LOADER)), # Resize 후 CenterCrop 또는 바로 사용
        # transforms.CenterCrop(IMG_SIZE_FOR_LOADER), # timm은 주로 Resize만 사용
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# --- 3. 모델 학습 및 예측 함수 ---
def train_one_epoch(model, loader, criterion, optimizer, device, grad_clip_value=None):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(loader, desc="학습 중", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        if grad_clip_value:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_value)
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

def get_oof_predictions_for_fold(model, loader, device):
    model.eval()
    fold_probs_list = []
    with torch.no_grad():
        # ImageFolder의 Subset을 사용할 때, Subset의 __getitem__은 (image, label)을 반환
        for images, _ in tqdm(loader, desc="OOF 예측 중", leave=False):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            fold_probs_list.append(probs.cpu().numpy())
    return np.concatenate(fold_probs_list, axis=0)

# --- 4. OOF 예측 생성 메인 로직 ---
if __name__ == "__main__":
    os.makedirs(OOF_OUTPUT_DIR, exist_ok=True)
    print(f"OOF 예측 및 레이블 저장 위치: {OOF_OUTPUT_DIR}")
    print(f"사용 디바이스: {DEVICE}")

    # 전체 학습 데이터셋 로드 (ImageFolder 사용, 변환은 나중에 적용)
    # ImageFolder는 초기화 시 transform을 받지만, Subset에 직접 적용되지 않으므로,
    # Subset을 만들고 DataLoader에 전달할 때 transform이 적용된 Dataset을 사용해야 함.
    # 또는, fold마다 ImageFolder를 새로 생성하고 transform을 전달.
    # 여기서는 먼저 전체 ImageFolder를 로드하여 레이블 정보와 클래스 순서를 얻음.
    
    # 임시 변환 없이 ImageFolder 로드 (클래스 및 레이블 정보 획득용)
    temp_full_train_dataset = datasets.ImageFolder(root=TRAIN_IMAGE_FOLDER_PATH, transform=None)
    
    # ImageFolder가 인식한 클래스 순서 확인 및 CLASS_NAMES와 일치시키기
    imagefolder_classes = temp_full_train_dataset.classes
    print(f"ImageFolder가 인식한 클래스: {imagefolder_classes}")
    if len(imagefolder_classes) != NUM_CLASSES:
        raise ValueError(f"ImageFolder 클래스 수({len(imagefolder_classes)})와 NUM_CLASSES({NUM_CLASSES}) 불일치")
    
    # CLASS_NAMES_EXPECTED 순서에 맞게 ImageFolder의 레이블을 재매핑할 필요는 없음.
    # ImageFolder는 자체적으로 0부터 시작하는 인덱스를 부여함.
    # 생성된 OOF 파일과 함께 이 imagefolder_classes 순서를 기억해두거나,
    # train_meta_models.py 에서 CLASS_NAMES_EXPECTED 를 기준으로 처리하면 됨.
    # 여기서는 imagefolder_classes를 최종 클래스 이름으로 사용한다고 가정.
    CLASS_NAMES_ACTUAL = imagefolder_classes # 실제 사용될 클래스 이름 순서

    all_true_labels = np.array(temp_full_train_dataset.targets) # ImageFolder의 레이블 (정수 인덱스)
    num_total_train_samples = len(all_true_labels)
    print(f"전체 학습 데이터 로드 완료 (샘플 수: {num_total_train_samples}, 클래스 수: {len(CLASS_NAMES_ACTUAL)})")

    # Stratified K-Fold 준비
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    # 전체 학습 데이터의 실제 레이블 저장 (train_meta_models.py 에서 사용)
    np.save(os.path.join(OOF_OUTPUT_DIR, 'train_labels.npy'), all_true_labels)
    # 클래스 이름 순서도 저장 (중요!)
    with open(os.path.join(OOF_OUTPUT_DIR, 'class_names_actual.txt'), 'w') as f:
        for class_name in CLASS_NAMES_ACTUAL:
            f.write(f"{class_name}\n")
    print(f"전체 학습 데이터 레이블 및 클래스 이름 저장 완료.")
    print(f"  train_labels.npy (형태: {all_true_labels.shape})")
    print(f"  class_names_actual.txt (내용: {CLASS_NAMES_ACTUAL})")


    # 각 기본 모델에 대해 OOF 예측 생성
    for model_config in MODEL_CONFIGS:
        print(f"\n--- 기본 모델 OOF 생성 시작: {model_config['name']} ---")
        model_oof_preds = np.zeros((num_total_train_samples, NUM_CLASSES))
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(X=np.zeros(num_total_train_samples), y=all_true_labels)):
            print(f"\n  Fold {fold+1}/{N_SPLITS} 처리 중...")

            # 현재 Fold의 학습 및 검증 Subset 생성 및 변환 적용
            # 각 Fold마다 ImageFolder를 transform과 함께 새로 생성하고 Subset으로 나눔
            # 이렇게 하면 Subset의 각 아이템에 transform이 올바르게 적용됨
            train_fold_dataset = datasets.ImageFolder(root=TRAIN_IMAGE_FOLDER_PATH, transform=train_transform)
            val_fold_dataset = datasets.ImageFolder(root=TRAIN_IMAGE_FOLDER_PATH, transform=val_transform)
            
            train_subset_for_loader = Subset(train_fold_dataset, train_idx)
            val_subset_for_loader = Subset(val_fold_dataset, val_idx)

            train_loader = DataLoader(train_subset_for_loader, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
            val_loader = DataLoader(val_subset_for_loader, batch_size=BATCH_SIZE_VAL, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
            
            print(f"  학습 데이터 수: {len(train_subset_for_loader)}, 검증 데이터 수: {len(val_subset_for_loader)}")

            # 모델 초기화
            model_args = {'pretrained': True, 'num_classes': NUM_CLASSES}
            if model_config.get("requires_img_size_arg", False):
                 model_args['img_size'] = model_config['img_size']
            
            model = timm.create_model(model_config['architecture'], **model_args)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            # AdamW 옵티마이저 사용 (ConvNeXt 권장)
            optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
            
            # (선택 사항) 학습률 스케줄러 (예: CosineAnnealingLR with Warmup)
            # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=EPOCHS_PER_FOLD // 2, T_mult=1, eta_min=LEARNING_RATE * 0.01)
            # Warmup 스케줄러는 직접 구현하거나 라이브러리 사용 필요

            print(f"  Fold {fold+1} 모델 학습 시작 (에포크: {EPOCHS_PER_FOLD})...")
            for epoch in range(EPOCHS_PER_FOLD):
                # Warmup (간단한 선형 Warmup 예시)
                # current_lr = LEARNING_RATE
                # if epoch < WARMUP_EPOCHS:
                #     current_lr = LEARNING_RATE * (epoch + 1) / WARMUP_EPOCHS
                # for param_group in optimizer.param_groups:
                #     param_group['lr'] = current_lr
                
                train_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE, GRADIENT_CLIPPING)
                # if scheduler:
                #     scheduler.step()
                print(f"    Epoch {epoch+1}/{EPOCHS_PER_FOLD}, Train Loss: {train_loss:.4f}") # , LR: {optimizer.param_groups[0]['lr']:.2e}


            fold_val_probs = get_oof_predictions_for_fold(model, val_loader, DEVICE)
            model_oof_preds[val_idx] = fold_val_probs
            
            print(f"  Fold {fold+1} OOF 예측 생성 완료. 형태: {fold_val_probs.shape}")
            del model, train_loader, val_loader
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        oof_filename = f"{model_config['name'].replace(' ', '_')}_oof_probs.npy"
        oof_save_path = os.path.join(OOF_OUTPUT_DIR, oof_filename)
        np.save(oof_save_path, model_oof_preds)
        print(f"\n모델 {model_config['name']}의 전체 OOF 예측 저장 완료: {oof_save_path}, 형태: {model_oof_preds.shape}")

    print("\n모든 기본 모델에 대한 OOF 예측 생성 완료.")

timm 기반 변환 사용. 학습: Compose(
    RandomResizedCropAndInterpolation(size=(384, 384), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.6, 1.4), contrast=(0.6, 1.4), saturation=(0.6, 1.4), hue=None)
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)
timm 기반 변환 사용. 검증: Compose(
    Resize(size=(384, 384), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(384, 384))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)
OOF 예측 및 레이블 저장 위치: ./oof_predictions_imgfolder
사용 디바이스: cuda
ImageFolder가 인식한 클래스: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']
전체 학습 데이터 로드 완료 (샘플 수: 342015, 클래스 수: 7)
전체 학습 데이터 레이블 및 클래스 이름 저장 완료.
  train_labels.npy (형태: (342015,))
  class_names_actual.txt (내용: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mu

KeyboardInterrupt: 